In [4]:
variable=10

function add_using_global_variable(x)
    return x+variable
end
using BenchmarkTools

@btime add_using_global_variable(10)

  24.095 ns (0 allocations: 0 bytes)


20

In [7]:
function add_using_function_args(x,y)
    return x+y
end

@btime add_using_function_args(10,10)

  0.001 ns (0 allocations: 0 bytes)


20

In [9]:
@code_llvm add_using_function_args(10,10)


;  @ In[7]:1 within `add_using_function_args'
; Function Attrs: uwtable
define i64 @julia_add_using_function_args_1576(i64, i64) #0 {
top:
;  @ In[7]:2 within `add_using_function_args'
; ┌ @ int.jl:86 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


In [11]:
@code_llvm add_using_global_variable(10)


;  @ In[4]:3 within `add_using_global_variable'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_add_using_global_variable_1578(i64) #0 {
top:
  %1 = alloca %jl_value_t*, i32 2
  %gcframe = alloca %jl_value_t*, i32 4, align 16
  %2 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %2, i8 0, i32 32, i1 false)
  %3 = call %jl_value_t*** inttoptr (i64 1720679008 to %jl_value_t*** ()*)() #6
;  @ In[4]:4 within `add_using_global_variable'
  %4 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %5 = bitcast %jl_value_t** %4 to i64*
  store i64 8, i64* %5
  %6 = getelementptr %jl_value_t**, %jl_value_t*** %3, i32 0
  %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %8 = bitcast %jl_value_t** %7 to %jl_value_t***
  %9 = load %jl_value_t**, %jl_value_t*** %6
  store %jl_value_t** %9, %jl_value_t*** %8
  %10 = bitcast %jl_value_t*** %6 to %jl_value_t***
  store %jl_value_t** %gcframe, %jl_value_t*** %10
  %11 = load %jl

In [17]:
function constant_folding_example()
    a=2*3
    b=a+1
    b>1 ? 10 : 20
end

constant_folding_example()

10

In [1]:
@code_typed constant_folding_example()

LoadError: UndefVarError: constant_folding_example not defined

In [3]:
random_data(n)=isodd(n) ? rand(Int,n) : rand(Float64,n)

function double_sum_of_random_data(n)
    data=random_data(n)
    total=0
    for v in data
        total+=2v
    end
    return total
end


@code_warntype double_sum_of_random_data(1000)


Variables
  #self#::Core.Compiler.Const(double_sum_of_random_data, false)
  n::Int64
  data::Union{Array{Float64,1}, Array{Int64,1}}
  total::Union{Float64, Int64}
  @_5::Union{Nothing, Tuple{Union{Float64, Int64},Int64}}
  v::Union{Float64, Int64}

Body::Union{Float64, Int64}
1 ─       (data = Main.random_data(n))
│         (total = 0)
│   %3  = data::Union{Array{Float64,1}, Array{Int64,1}}
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_5::Tuple{Union{Float64, Int64},Int64}::Tuple{Union{Float64, Int64},Int64}
│         (v = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = total::Union{Float64, Int64}
│   %12 = (2 * v)::Union{Float64, Int64}
│         (total = %11 + %12)
│         (@_5 = Base.iterate(%3, %10))
│   %15 = (@_5 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return total


In [5]:
function double_sum(data)
    total=0
    for v in data
        total+=2v
    end
    return total
end

function double_sum_of_random_data(n)
    data=randon_data(n)
    return double_sum(n)
end

@code_warntype double_sum(rand(Int,3))

Variables
  #self#::Core.Compiler.Const(double_sum, false)
  data::Array{Int64,1}
  total::Int64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  v::Int64

Body::Int64
1 ─       (total = 0)
│   %2  = data::Array{Int64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (v = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = total::Int64
│   %11 = (2 * v)::Int64
│         (total = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return total


In [6]:
@code_warntype double_sum(rand(Float64,3))

Variables
  #self#::Core.Compiler.Const(double_sum, false)
  data::Array{Float64,1}
  total::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  v::Float64

Body::Union{Float64, Int64}
1 ─       (total = 0)
│   %2  = data::Array{Float64,1}
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│         (v = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = total::Union{Float64, Int64}
│   %11 = (2 * v)::Float64
│         (total = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return total
